# Extract HOG Features

In [1]:
import os
import numpy as np
import cv2
from sklearn import svm 
import numpy as np 
from skimage.feature import hog 
trainKeyposints = []
trainDescriptors = []
labels = []
orb = cv2.ORB_create()

hog_train_data = [] 
# hog = cv2.HOGDescriptor()


for folder in os.listdir("dataset/train/"):
    print("Processing Folder = ",folder)
    
    for file in os.listdir("dataset/train/" + folder + "/"):

        name = 'dataset/train/'+ folder +"/" + file

        train_img = cv2.imread('dataset/train/'+ folder +"/" + file)
       
        if train_img is not None:

            
            
            dim = (32, 32)

            # resize image
            img = cv2.resize(train_img, dim, interpolation = cv2.INTER_AREA)

    #         here fd is the new feature descriptor for your image 
            fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)
    #         for visualisation of the hog features 
    #         plt.imshow(hog_img) 
            hog_train_data.append(fd) 

    #         train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)


    #         Keypoints, Descriptors = orb.detectAndCompute(train_img_bw,None)
    #         if Descriptors is not None:
    #             print(Descriptors.shape)
    #             trainDescriptors.append(Descriptors)#.flatten())
    #             trainKeyposints.append(Keypoints)
            labels.append(int(folder))
    # #             print(len(Descriptors.flatten()))
    #         else: 
    #             print("NONE Descriptor")
    #             pass
        else:
            print("Error Opening image")
        pass


Processing Folder =  00004
Processing Folder =  00002
Processing Folder =  00006
Processing Folder =  00001
Processing Folder =  00007
Processing Folder =  00008
Processing Folder =  00003
Processing Folder =  00005
Processing Folder =  00010
Processing Folder =  00009


# Train SVM Classifier

In [2]:
#convert it to np array 
hog_train_data = np.array(hog_train_data) 
#one-vs-all SVM linear classifier 
lin_clf = svm.LinearSVC() 
#train the classifier using new hog features and training labels 
lin_clf.fit(hog_train_data, labels) 
#likewise extract features for your testing data and test the classifier using the new features 

LinearSVC()

# Testing trained SVM on test images

In [19]:
dirct = "dataset/test/"


tp = 0 # True Positive
fp = 0 # False Positive
fn = 0 # False Negative


for folder in os.listdir(dirct):
    print("Processing Folder = ",folder)
    
    for file in os.listdir(dirct + folder + "/"):

        name = dirct+ folder +"/" + file

        test_img = cv2.imread(dirct+ folder +"/" + file)
       
        if test_img is not None:

            
            
            dim = (32, 32)

            # resize image
            img = cv2.resize(test_img, dim, interpolation = cv2.INTER_AREA)

    #         here fd is the new feature descriptor for your image 
            fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)

            pred = lin_clf.predict([fd])
            print("Pred = ",pred[0])
            print("Label = ", int(folder))
            if pred[0] == int(folder):
                tp += 1
            else:
                fp += 1
                fn += 1


        else:
            print("Error Opening image")
        pass

Processing Folder =  00004
Pred =  4
Label =  4
Pred =  4
Label =  4
Pred =  4
Label =  4
Pred =  4
Label =  4
Pred =  4
Label =  4
Pred =  4
Label =  4
Processing Folder =  00002
Pred =  2
Label =  2
Pred =  2
Label =  2
Pred =  2
Label =  2
Pred =  2
Label =  2
Pred =  2
Label =  2
Pred =  2
Label =  2
Processing Folder =  00006
Pred =  6
Label =  6
Pred =  6
Label =  6
Pred =  6
Label =  6
Pred =  6
Label =  6
Pred =  6
Label =  6
Processing Folder =  00001
Pred =  1
Label =  1
Pred =  1
Label =  1
Pred =  1
Label =  1
Pred =  1
Label =  1
Pred =  1
Label =  1
Pred =  1
Label =  1
Processing Folder =  00007
Pred =  7
Label =  7
Pred =  7
Label =  7
Pred =  7
Label =  7
Pred =  7
Label =  7
Pred =  7
Label =  7
Processing Folder =  00008
Pred =  8
Label =  8
Pred =  8
Label =  8
Pred =  8
Label =  8
Pred =  8
Label =  8
Pred =  8
Label =  8
Processing Folder =  00003
Pred =  3
Label =  3
Pred =  3
Label =  3
Pred =  3
Label =  3
Pred =  3
Label =  3
Pred =  3
Label =  3
Pred =  3
Lab

# Performance Measures

## Precision

In [14]:
precision = tp/(tp+fp)
print("Precision = ", precision)

Precision =  0.9821428571428571


In [16]:
recall = tp/(tp + fn)
print("Recall = ", recall)

Recall =  0.9821428571428571


In [18]:
f_1 = 2*(precision * recall)/(precision + recall)
print("F_1 Score = ", f_1)

F_1 Score =  0.9821428571428571
